# ETL_Credits

In [1]:
#Primero importaremos todas las herramientas que utilizaremos en el proceso de ETL

import ast 
import pandas as pd


In [2]:
#Cargamos los datasets en variables con el metodo de pandas pd.read_csv()
credits_df = pd.read_csv('/Users/pabloclementi/Desktop/Proyecto1/Proyecto_final_1/data_original/credits.csv')


In [5]:
#Crear credits_df_cast con las columnas "cast" y "id"
credits_df_cast = credits_df[['cast', 'id']].copy()

#Crear credits_df_crew con las columnas "crew" y "id"
credits_df_crew = credits_df[['crew', 'id']].copy()


In [6]:
#Realizamos una funcion para eliminar espacios en blanco en las columnas
def trim_columnas(df):
    for col in df.select_dtypes(include=['object']):
        df[col] = df[col].apply(lambda x: x.strip() if isinstance(x, str) else x)
    return df

In [7]:
credits_df_cast=trim_columnas(credits_df_cast)
credits_df_crew=trim_columnas(credits_df_crew) 
# credits_df_cast.head()
# credits_df_crew.head()
# credits_df_cast.info()
# credits_df_crew.info() #Se verifico que no hay duplicados haciendo un drop duplicate la info sigue igual. #Borramos la linea de codigo que eliminaba duplicados.

In [8]:
credits_df_crew.columns

Index(['crew', 'id'], dtype='object')

In [9]:
credits_df_cast.columns

Index(['cast', 'id'], dtype='object')

In [10]:
columns_to_expand_crew = ['crew'] 
columns_to_expand_cast = ['cast']


In [11]:
def expandircolumnas(df, columns):
    for column in columns:
        df[column] = df[column].apply(lambda x: ast.literal_eval(x) if pd.notnull(x) else [])
        # Expandir la columna en filas
        df = df.explode(column)
        # Normalizar la columna expandida
        col_df = pd.json_normalize(df[column])
        # Renombrar las columnas normalizadas para evitar superposiciones
        col_df = col_df.add_prefix(f'{column}')
        # Concatenar las columnas normalizadas con el DataFrame original
        df = df.drop(columns=[column]).reset_index(drop=True).join(col_df)
    return df



In [12]:
credits_df_crew = expandircolumnas(credits_df_crew, columns_to_expand_crew)
credits_df_cast = expandircolumnas(credits_df_cast, columns_to_expand_cast) 
credits_df_crew.head()


,id,crewcredit_id,crewdepartment,crewgender,crewid,crewjob,crewname,crewprofile_path
0,862,52fe4284c3a36847f8024f49,Directing,2.0,7879.0,Director,John Lasseter,/7EdqiNbr4FRjIhKHyPPdFfEEEFG.jpg
1,862,52fe4284c3a36847f8024f4f,Writing,2.0,12891.0,Screenplay,Joss Whedon,/dTiVsuaTVTeGmvkhcyJvKp2A5kr.jpg
2,862,52fe4284c3a36847f8024f55,Writing,2.0,7.0,Screenplay,Andrew Stanton,/pvQWsu0qc8JFQhMVJkTHuexUAa1.jpg
3,862,52fe4284c3a36847f8024f5b,Writing,2.0,12892.0,Screenplay,Joel Cohen,/dAubAiZcvKFbboWlj7oXOkZnTSu.jpg
4,862,52fe4284c3a36847f8024f61,Writing,0.0,12893.0,Screenplay,Alec Sokolow,/v79vlRYi94BZUQnkkyznbGUZLjT.jpg


In [13]:
#De este Dataframe mantendremos la columna id, la columna crewjob == 'Director'
df_directores = credits_df_crew[credits_df_crew['crewjob'] == 'Director']

# Seleccionar solo las columnas 'id' y 'crewname' (renombrada como 'director')
df_directores = df_directores[['id', 'crewname']].copy()

# Renombrar la columna 'crewname' a 'director'
df_directores.rename(columns={'crewname': 'director'}, inplace=True)
#Reseteamos index 
df_directores = df_directores.reset_index(drop=True)
# Mostrar el nuevo DataFrame con los directores
df_directores.head()

,id,director
0,862,John Lasseter
1,8844,Joe Johnston
2,15602,Howard Deutch
3,31357,Forest Whitaker
4,11862,Charles Shyer


In [14]:
credits_df_cast.head()

,id,castcast_id,castcharacter,castcredit_id,castgender,castid,castname,castorder,castprofile_path
0,862,14.0,Woody (voice),52fe4284c3a36847f8024f95,2.0,31.0,Tom Hanks,0.0,/pQFoyx7rp09CJTAb932F2g8Nlho.jpg
1,862,15.0,Buzz Lightyear (voice),52fe4284c3a36847f8024f99,2.0,12898.0,Tim Allen,1.0,/uX2xVf6pMmPepxnvFWyBtjexzgY.jpg
2,862,16.0,Mr. Potato Head (voice),52fe4284c3a36847f8024f9d,2.0,7167.0,Don Rickles,2.0,/h5BcaDMPRVLHLDzbQavec4xfSdt.jpg
3,862,17.0,Slinky Dog (voice),52fe4284c3a36847f8024fa1,2.0,12899.0,Jim Varney,3.0,/eIo2jVVXYgjDtaHoF19Ll9vtW7h.jpg
4,862,18.0,Rex (voice),52fe4284c3a36847f8024fa5,2.0,12900.0,Wallace Shawn,4.0,/oGE6JqPP2xH4tNORKNqxbNPYi7u.jpg


In [15]:
#Aplicamos el mismo proceso para los actores en el dataframe credits_df_cast, solo obteniendo id y castname 
df_actores = credits_df_cast[['id', 'castname']].copy()
df_actores.rename(columns={'castname': 'actor'}, inplace=True)
df_actores = df_actores.reset_index(drop=True)
df_actores.head()

#Unimos los dataframes df_directores y df_actores 
df_directores_actores = pd.merge(df_directores, df_actores, on='id', how='left')







In [16]:
df_directores_actores.head()

,id,director,actor
0,862,John Lasseter,Tom Hanks
1,862,John Lasseter,Tim Allen
2,862,John Lasseter,Don Rickles
3,862,John Lasseter,Jim Varney
4,862,John Lasseter,Wallace Shawn


In [17]:
#Convertimos a string las columnas actor y directores
df_directores_actores.info()
df_directores_actores['actor'] = df_directores_actores['actor'].astype(str)
df_directores_actores['director'] = df_directores_actores['director'].astype(str)
df_directores_actores.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 615750 entries, 0 to 615749
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        615750 non-null  int64 
 1   director  615750 non-null  object
 2   actor     613363 non-null  object
dtypes: int64(1), object(2)
memory usage: 14.1+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 615750 entries, 0 to 615749
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        615750 non-null  int64 
 1   director  615750 non-null  object
 2   actor     615750 non-null  object
dtypes: int64(1), object(2)
memory usage: 14.1+ MB


In [18]:
# Agrupar por ID de película y concatenar los actores
df_agrupado = df_directores_actores.groupby('id').agg({
    'director': 'first',  # Tomar el primer valor de la columna 'director'
    'actor': lambda x: ', '.join(x)  # Concatenar los valores de la columna 'actor'
}).reset_index()

In [19]:
df_agrupado.head()

,id,director,actor
0,2,Aki Kaurismäki,"Turo Pajala, Susanna Haavisto, Matti Pellonpää..."
1,3,Aki Kaurismäki,"Matti Pellonpää, Kati Outinen, Sakari Kuosmane..."
2,5,Allison Anders,"Tim Roth, Antonio Banderas, Jennifer Beals, Ma..."
3,6,Stephen Hopkins,"Emilio Estevez, Cuba Gooding Jr., Denis Leary,..."
4,11,George Lucas,"Mark Hamill, Harrison Ford, Carrie Fisher, Pet..."


In [20]:
#Verificamos si hay duplicados 
df_agrupado.duplicated().sum()


0

In [21]:
#Exportamos el df_agrupado como archivo csv con el nombre credits_Limpio.csv 
df_agrupado.to_csv('credits_Limpio.csv', index=False)